# Intuit Quickbooks Upgrade

* Team-lead GitLab userid:
* Group name:
* Team member names:

## Setup

Please complete this python notebook with your group by answering the questions in `intuit.pdf`. Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML file from the Jupyter Notebook without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the first group assignment for MGTA 455 and you will be using Git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". This is not something serious but you should realize that Git will not decide for you who's change to accept so the team-lead will have to determine the edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitLab. Make "pull first" a habit!

If multiple people are going to work on the assignment at the same time I recommend you work in different notebooks. You can then `%run ...`  these "sub" notebooks from the main assignment file. You can seen an example of this in action below for the `question1.ipynb` notebook

Some group work-flow tips:

* Pull, edit, save, stage, commit, and push
* Schedule who does what and when
* Try to avoid working simultaneously on the same file 
* If you are going to work simultaneously, do it in different notebooks, e.g., 
    - question1.ipynb, question2.ipynb, question3.ipynb
* Use the `%run ... ` command to bring different pieces of code together into the main jupyter notebook
* Put python functions in modules that you can import from your notebooks. See the example below for the `example` function defined in `utils/functions.py`

A graphical depiction of the group work-flow is shown below:

![](images/git-group-workflow-wbg.png)

Tutorial videos about using Git, GitLab, and GitGadget for group assignments:

* Setup the MSBA server to use Git and GitLab: https://youtu.be/zJHwodmjatY
* Dealing with Merge Conflicts: https://youtu.be/qFnyb8_rgTI
* Group assignment practice: https://youtu.be/4Ty_94gIWeA

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [2]:
## loading the data - this dataset must NOT be changed
intuit75k = pd.read_pickle("/home/jovyan/git/rsm-mgta455-intuit/data/intuit75k.pkl")
intuit75k["res1_yes"] = (intuit75k["res1"] == "Yes").astype(int)
intuit75k.head()

,id,zip,zip_bins,sex,bizflag,numords,dollars,last,sincepurch,version1,owntaxprod,upgraded,res1,training,res1_yes
0,1,94553,18,Male,0,2,109.5,5,12,0,0,0,No,1,0
1,2,53190,10,Unknown,0,1,69.5,4,3,0,0,0,No,0,0
2,3,37091,8,Male,0,4,93.0,14,29,0,0,1,No,0,0
3,4,02125,1,Male,0,1,22.0,17,1,0,0,0,No,1,0
4,5,60201,11,Male,0,1,24.5,2,3,0,0,0,No,0,0


In [3]:
intuit75k['upgraded']= pd.Categorical(intuit75k['upgraded'])

In [4]:
# show dataset description
rsm.describe(intuit75k)

## Intuit: Quickbooks upgrade

The purpose of this exercise is to gain experience modeling the response to an upsell campaign. The `intuit75k.rds` file contains data on 75,000 (small) businesses that were selected randomly from the 801,821 that were sent the wave-1 mailing. The mailing contained an offer to upgrade to the latest version of the Quickbooks software. 

Variable `res1` denotes which of these businesses responded to the mailing by purchasing Quickbooks version 3.0 from Intuit Direct. Note that Intuit Direct sells products directly to its customers rather than through a retailer. Use the available data to predict which businesses that did not respond to the wave-1 mailing, are most likely to respond to the wave-2 mailing. Note that variables were added, deleted, and recoded so please ignore the variable descriptions in Exhibit 3 in the case in the course reader. Instead, use the variable descriptions below:

## Variable description

* id: Small business customer ID
* zip: 5-

In [5]:
##RFM Independent Bins
intuit75k = intuit75k.assign(rec_iq=rsm.xtile(intuit75k["last"], 5))
intuit75k = intuit75k.assign(freq_iq=rsm.xtile(intuit75k["numords"], 5, rev=True))
intuit75k = intuit75k.assign(mon_iq=rsm.xtile(intuit75k["dollars"], 5, True))
intuit75k = intuit75k.assign(
    rfm_iq=intuit75k["rec_iq"].astype(str)
    + intuit75k["freq_iq"].astype(str)
    + intuit75k["mon_iq"].astype(str)
)

In [6]:
#RFM Sequential Bins
intuit75k = intuit75k.assign(freq_sq=intuit75k.groupby("rec_iq")["numords"].transform(rsm.xtile, 5, rev=True))
intuit75k = intuit75k.assign(mon_sq=intuit75k.groupby(["rec_iq", "freq_sq"])["dollars"].transform(rsm.xtile, 5, rev=True))
intuit75k = intuit75k.assign(
    rfm_sq=intuit75k["rec_iq"].astype(str)
    + intuit75k["freq_sq"].astype(str)
    + intuit75k["mon_sq"].astype(str)
)

In [39]:
##Training dataset
intuit_training= intuit75k[intuit75k['training']==1]
intuit_test= intuit75k[intuit75k['training']==0]

,id,zip,zip_bins,sex,bizflag,numords,dollars,last,sincepurch,version1,...,res1,training,res1_yes,rec_iq,freq_iq,mon_iq,rfm_iq,freq_sq,mon_sq,rfm_sq
1,2,53190,10,Unknown,0,1,69.5,4,3,0,...,No,0,0,1,5,3,153,5,2,152
2,3,37091,8,Male,0,4,93.0,14,29,0,...,No,0,0,3,1,2,312,1,5,315
4,5,60201,11,Male,0,1,24.5,2,3,0,...,No,0,0,1,5,5,155,5,4,154
6,7,22980,5,Male,0,1,49.5,13,36,1,...,No,0,0,2,5,4,254,5,2,252
8,9,34950,8,Male,0,1,44.5,15,4,0,...,No,0,0,3,5,4,354,5,3,353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74981,74982,90803,16,Male,1,1,129.5,12,16,0,...,No,0,0,2,5,2,252,5,1,251
74982,74983,71446,12,Male,0,2,22.0,25,1,0,...,No,0,0,4,3,5,435,3,5,435
74987,74988,80501,15,Female,0,4,94.5,7,23,1,...,No,0,0,2,1,2,212,1,5,215
74993,74994,40223,8,Female,0,5,170.5,9,20,0,...,Yes,0,1,2,1,1,211,1,3,213


In [31]:
#Response
intuit_training['res1_yes'].agg(n_mails= "count",n_responders=sum)

n_mails         52500
n_responders     2498
Name: res1_yes, dtype: int64

In [32]:
#Response Rate
resp_rate= (np.nanmean(intuit_training['res1_yes'] == 1)*100).round(2).astype(str)
print("The percentage of customers who responded to wave_1 emails is " + resp_rate + "%")

The percentage of customers who responded to wave_1 emails is 4.76%


In [33]:
#Breakeven analysis
cost= 1.41
margin= 60
breakeven_rr= cost/margin
print("The breakeven response rate for wave_1 emails is "+ str(round(breakeven_rr*100,2))+ "%")

The breakeven response rate for wave_1 emails is 2.35%


In [49]:
# Find cells to target on test dataset based on breakeven
groups_iq= intuit_training.groupby("rfm_iq")["res1_yes"].agg((lambda x: (np.nanmean(x == 1))/2)).reset_index()
target_iq=groups_iq[groups_iq['res1_yes']>breakeven_rr]['rfm_iq']
groups_sq= intuit_training.groupby("rfm_sq")["res1_yes"].agg((lambda x: (np.nanmean(x == 1))/2)).reset_index()
target_sq=groups_sq[groups_sq['res1_yes']>breakeven_rr]['rfm_sq']

In [48]:
#RFM IQ Analysis
notify= intuit_test[intuit_test["rfm_iq"].isin(target_iq)]
nr_notify = notify.shape[0]
perc_notify= nr_notify/22500
nr_resp = notify[notify['res1_yes']==1].shape[0]
rep_rate= nr_resp/nr_notify
marginal_cost = 1.41 * nr_notify
revenue = nr_resp*60
profit_nt = revenue - marginal_cost
ROME_nt = profit_nt / marginal_cost
print(f"""Intuit would send push notifications to {int(nr_notify):,} ({round((100 * perc_notify), 2)}%) customers. The response rate for the selected customers is {round((100 * rep_rate), 2)}% or {int(nr_resp):,} buyers. The revenue is equal to ${int(revenue):,}. The profit is ${int(profit_nt):,}. The marginal cost is ${int(marginal_cost):,} with a ROME of {round((100 * ROME_nt), 2)}%""")

Intuit would send push notifications to 9,235 (41.04%) customers. The response rate for the selected customers is 7.56% or 698 buyers. The revenue is equal to $41,880. The profit is $28,858. The marginal cost is $13,021 with a ROME of 221.63%


In [50]:
#RFM SQ Analysis
notify= intuit_test[intuit_test["rfm_sq"].isin(target_sq)]
nr_notify = notify.shape[0]
perc_notify= nr_notify/22500
nr_resp = notify[notify['res1_yes']==1].shape[0]
rep_rate= nr_resp/nr_notify
marginal_cost = 1.41 * nr_notify
revenue = nr_resp*60
profit_nt = revenue - marginal_cost
ROME_nt = profit_nt / marginal_cost
print(f"""Intuit would send push notifications to {int(nr_notify):,} ({round((100 * perc_notify), 2)}%) customers. The response rate for the selected customers is {round((100 * rep_rate), 2)}% or {int(nr_resp):,} buyers. The revenue is equal to ${int(revenue):,}. The profit is ${int(profit_nt):,}. The marginal cost is ${int(marginal_cost):,} with a ROME of {round((100 * ROME_nt), 2)}%""")

Intuit would send push notifications to 9,272 (41.21%) customers. The response rate for the selected customers is 7.56% or 701 buyers. The revenue is equal to $42,060. The profit is $28,986. The marginal cost is $13,073 with a ROME of 221.72%
